In [133]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [299]:
%%time
import pandas as pd
from sklearn import metrics,cluster
import numpy as np
import random

#### Reading the feature and calculating the pairwise distance between all the points

df = pd.read_csv('gh_cellulase_PAAC.txt', sep='\t')# input file which contains features, header=None if no column title
x=df.iloc[:,1:].values

# distance=metrics.pairwise.pairwise_distances(x)

def cal_dist(b,c):
    temp=np.zeros([len(x),len(b)])
    for i1 in range(len(x)):
        for j1 in b:
            temp_dis=np.sqrt(sum([(xx-yy)**2 for xx,yy in zip(x[i1],c[j1])]))
            temp[i1][j1]=temp_dis
    return temp

#### collecting intra and inter distance for clusters


class clust:
    def __init__(self,ref,comp):
        self.ref=ref
        self.comp=comp
        if self.ref==self.comp:
#             print(f'The intra cluster distance within cluster {self.ref}')
            self.c=1
        else:
#             print(f'The inter cluster distance between cluster {self.ref} and {self.comp}')
            self.c=0
        
    def all_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            for y1 in range((x1+1)*self.c,len(labels)):
                if labels[x1]==self.ref and labels[y1]==self.comp:
#                     print(f'{x1}..{y1}',distance[x1][y1])
                    al_data.append(distance[x1][y1])
        self.data=al_data
        self.avg=sum(al_data)/len(al_data)
        return self

    def centroid_dist(self):
        al_data=[]
        for x1 in range(len(labels)):
            if labels[x1]==self.ref:
                al_data.append(cent_dist[x1][self.comp])
        self.data=al_data
        self.avg=sum(al_data)/len(al_data)
        return self               

def un_learn(n):
    kmeans = cluster.KMeans(n_clusters=n).fit(x) # Number of clusters
    lab = kmeans.labels_
    centroid=kmeans.cluster_centers_
    cent_lab=kmeans.predict(centroid)
    c_d=cal_dist(cent_lab,centroid)
    return lab,centroid,c_d

labels,cent,cent_dist=un_learn(3)
a=clust(0,0).centroid_dist() # cluster compared to centroid

Wall time: 47 ms


In [300]:
%%time
from collections import Counter
from itertools import combinations
import numpy as np

# a=clust(0,1).all_dist()
# b=clust(0,0).all_dist()
##########################################################################
nn=3 # number of cluster
ni=100 # number of iterations
##########################################################################
lis=[s for s in range(nn)]
grp=list(combinations(lis,2))
grp.extend([(i,i) for i in lis])

grp_taken=grp

min_dis,max_dis,avg_dis={},{},{}
for each in grp_taken:
    min_dis[each],max_dis[each],avg_dis[each]=[],[],[]
    
ll=[]
for each_iter in range(ni):
#     print(f'Iteration:{each_iter+1}')
#     labels,no_lab=un_learn(nn)
    labels,cent,cent_dist=un_learn(nn)
    ll.append(labels)
    for j in grp_taken:
#         c=clust(j[0],j[1]).all_dist()
        c=clust(j[0],j[0]).centroid_dist() # cluster compared to centroid
        min_dis[j].append(min(c.data))
        max_dis[j].append(max(c.data))
        avg_dis[j].append(c.avg)

main_list_max=[]
main_list_min=[]
for each_iter in range(ni):
#     print(f'Iteration:{each_iter+1}')
    emp_array_avg=np.zeros((len(cent),len(cent)))
    emp_array_m=np.zeros((len(cent),len(cent)))
    sub_list=[]
    for k in grp_taken:
        if k[0]==k[1]:
            emp_array_m[k[0]][k[1]]=max_dis[k][each_iter]
            sub_list.append(max_dis[k][each_iter])
    main_list_max.append(max(sub_list))# maximum of maximum intra cluster distance from centroid
    main_list_min.append(min(sub_list))# minimum of maximum intra cluster distance from centroid
#         print(min(sub_list))
#         else:
#             emp_array_m[k[1]][k[0]]=min_dis[k][each_iter]
#             emp_array_m[k[0]][k[1]]=min_dis[k][each_iter]
# df1=pd.DataFrame(emp_array_m)
# df1

Wall time: 3.62 s


In [303]:
cx,ci=dict(Counter(main_list_max)),dict(Counter(main_list_min))
print('max:',cx)# lyso max
print('min:',ci)# lyso min

max: {0.030951986995242247: 54, 0.03111538226367291: 17, 0.031511899096967076: 7, 0.031077890787835655: 7, 0.031151143432181918: 1, 0.030978424498234697: 2, 0.0322556332187436: 1, 0.031261463267333145: 1, 0.03148044151015463: 2, 0.0316331895658817: 1, 0.031168096400549697: 1, 0.03162207502437474: 1, 0.031104210859222253: 3, 0.03274791490624315: 1, 0.03119866361442764: 1}
min: {0.029516337953961497: 56, 0.02424619302646035: 27, 0.023761131912604774: 6, 0.02341369943100249: 2, 0.023129977990335115: 1, 0.022658981546510092: 1, 0.02334451044011748: 4, 0.02191390861299122: 1, 0.0231610710582992: 1, 0.020343681474981584: 1}


In [292]:
%store -r ll_dist

array([2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [269]:
Counter(main_list)# H1N1 maximum

Counter({0.02272888530397871: 100})

In [271]:
Counter(main_list)# H1N1 minimum

Counter({0.006620163628200178: 25,
         0.006620163628200176: 71,
         0.006620163628200175: 4})

In [272]:
c=[]
for ii in range(len(main_list)):
    if main_list[ii]==0.006620163628200178:
        c.append(ll[ii])
print(len(c))
d=[]
for ii in range(len(main_list)):
    if main_list[ii]==0.006620163628200176:
        d.append(ll[ii])
print(len(d))

25
71


In [266]:
from itertools import combinations
aa=list(combinations([z for z in range(6)],2))
for aaa in aa:
    print(aaa)
    a=[]
    for x,y in zip(c[aaa[0]],c[aaa[1]]):
        a.append(f'{x}{y}')
    
    if len(dict(Counter(a)))==3:
        print(dict(Counter(a)))
    else:
        print('Alert!!!!!!!!!!!!')

(0, 1)
{'11': 16, '22': 37, '00': 59}
(0, 2)
{'12': 16, '21': 37, '00': 59}
(0, 3)
{'12': 16, '20': 37, '01': 59}
(0, 4)
{'10': 16, '21': 37, '02': 59}
(0, 5)
{'12': 16, '21': 37, '00': 59}
(1, 2)
{'12': 16, '21': 37, '00': 59}
(1, 3)
{'12': 16, '20': 37, '01': 59}
(1, 4)
{'10': 16, '21': 37, '02': 59}
(1, 5)
{'12': 16, '21': 37, '00': 59}
(2, 3)
{'22': 16, '10': 37, '01': 59}
(2, 4)
{'20': 16, '11': 37, '02': 59}
(2, 5)
{'22': 16, '11': 37, '00': 59}
(3, 4)
{'20': 16, '01': 37, '12': 59}
(3, 5)
{'22': 16, '01': 37, '10': 59}
(4, 5)
{'02': 16, '11': 37, '20': 59}


In [297]:
a=[]
for x,y in zip(ll[0],ll_dist[0]):
        a.append(f'{x}{y}')
    
if len(dict(Counter(a)))==3:
    print(dict(Counter(a)))
else:
    print(dict(Counter(a)))
    print('Alert!!!!!!!!!!!!')

{'22': 19, '01': 57, '21': 5, '10': 61, '20': 1}
Alert!!!!!!!!!!!!


In [257]:
Counter(main_list)# hemoglobin

Counter({0.023653583339059068: 81,
         0.023479456571733056: 6,
         0.023664171775432848: 13})